<a href="https://colab.research.google.com/github/heejininku/class2022Fall/blob/main/20221118(3)_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading a dataset

In [ ]:
import pandas as pd

### computer hardware dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #이 le 함수 필요해서 가져옴
computer_hardware = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data', header=None, names=['vendor', 'model', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'label'])
#read_csv 함수를 통해서 csv 가져옴.열어보면 숫자만 가득 (헤더, 칼럼명 없이)
#header=None이라고 쓰고 없기에 따라서 헤더명을 뒤에 적음.
computer_hardware['vendor'] = le.fit_transform(computer_hardware['vendor']) 
computer_hardware['model'] = le.fit_transform(computer_hardware['model'])
#vendor, model이 숫자가 아니니까 숫자화해준 것. 위에 두 줄이.
computer_hardware.head() #데이터프레임현태 (pandas에서 csv하니까?)

,vendor,model,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,label
0,0,29,125,256,6000,256,16,128,198,199
1,1,62,29,8000,32000,32,8,32,269,253
2,1,63,29,8000,32000,32,8,32,220,253
3,1,64,29,8000,32000,32,8,32,172,253
4,1,65,29,8000,16000,32,8,16,132,132


In [ ]:
computer_hardware
#나중에 label이라는 부분을 출력값으로 쓸 것임. 앞에 있는 입력은 9개
#출력부분은 진정한 의미의 숫자여야함.

,vendor,model,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,label
0,0,29,125,256,6000,256,16,128,198,199
1,1,62,29,8000,32000,32,8,32,269,253
2,1,63,29,8000,32000,32,8,32,220,253
3,1,64,29,8000,32000,32,8,32,172,253
4,1,65,29,8000,16000,32,8,16,132,132
...,...,...,...,...,...,...,...,...,...,...
204,27,100,124,1000,8000,0,1,8,42,37
205,27,109,98,1000,8000,32,2,8,46,50
206,28,28,125,2000,8000,0,2,14,52,41
207,29,207,480,512,8000,32,0,0,67,47


### parkinsons dataset

In [ ]:
parkinsons = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/telemonitoring/parkinsons_updrs.data')
#불러오고
parkinsons.rename(columns={'motor_UPDRS':'label'}, inplace=True)
#rename을 해준 것.(통일하도록 label이라는 이름으로)
parkinsons.head()

,subject#,age,sex,test_time,label,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


## Comparing Models

classification, regression의 목적을 알자.
예측.prediction을 위한 것.
이제 데이터보다 y=ax+b 선을 가지고 있으면 되니까

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

models = [
    ('LR', LinearRegression()), #이거 설명 놓침
    ('L', Lasso()),
    ('R', Ridge()),
    ('SVR', LinearSVR()),
    ('DT', DecisionTreeRegressor()),
    ('NN', MLPRegressor()), ] #딥러닝

이런 애들 중 어떤 게 좋은지 평가해야함. 평가하는 방법 아래에    

데이터 중에서 80프로는 모델만드는데 쓰고 20프로는 잘 되나 평가하는 거.

### test a single model (e.g. Linear Regression)

가장 대표적인 linear regression을 먼저 해보겠음.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

dataset_name = 'computer_hardware'
dataset = computer_hardware

#하나는 x,하나는 y로 데이터 속에서 빼옴.
X = np.array(dataset.drop(['label'], axis=1))  
#label안에 있는 값 (칼럼 통째로) 다 빼라
#데이터프레임의 형태를 np로 바꾸어줘야함.
y = np.array(dataset['label'])  #y값은 label #이것도 numpy로 바꿔준 것
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#테스트할 것을 0.2 (20프로), 변수를 총 네개로
#두개는 훈련용, 두개는 테스트용


name ='LR'
model = LinearRegression() 
model.fit(X_train, y_train) #.fit은 데이터를 관통하기 위해 계속 찾아내는 것.y=ax+b에서 a,b값을 찾아주는 것
accuracy = model.score(X_test, y_test) #테스트하기
print(dataset_name, name, accuracy) #값이 0.94...이렇게 나오면 94프로 맞는 것.

computer_hardware LR 0.9438611618299354


아래에는 9개의 입력값을 리스트로 만든 것.
근데 nd.array로 받는 것
reshape해서 (1,9) 즉. 1개의 행, 9개의 열
(아니면 그냥 리스트 꺽세를 두개해서 2d로 만들어도 된다)
(여기서 1d,2d관련 이야기를 하셨는데 이해 못함.2d여야 하는 이유)

In [ ]:
model.predict(np.array([22,   122,    40,  8000, 16000,    32,     8,    16, 214]).reshape(1,9))

array([181.53279469])

### Evaluate all the models

데이터를 두가지 종류를 씀
하드웨어랑 팜킨슨
데이터 두개 평가해보고
6개의 모델들도 평각해보자
위에 loof에는 데이터셋
아래 loof에는 6개 모델
총 2x6 해서 12개의 값이 나올 것.
(어떤 순서로 루프를 도는지는 각자 살펴보기)

In [ ]:
for dataset_name, dataset in [('computer_hardware', computer_hardware), ('parkinsons', parkinsons)]:
    X = np.array(dataset.drop(['label'], axis=1))
    y = np.array(dataset['label'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for name, model in models:
        model.fit(X_train, y_train)
        accuracy = model.score(X_test, y_test)
        print(dataset_name, name, accuracy)

computer_hardware LR 0.9438611618299354
computer_hardware L 0.9439215152431056
computer_hardware R 0.9438612466623207
computer_hardware SVR 0.5913496750263022
computer_hardware DT 0.874488011234236
computer_hardware NN -9.414130858182011
parkinsons LR 0.9068547520909843
parkinsons L 0.8983382060322662
parkinsons R 0.9049789120180208


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


parkinsons SVR 0.8858833094338023
parkinsons DT 0.9988242332006754
parkinsons NN 0.9703160844438825


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
